In [17]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from rbf_neural_networks import RBFNeuralNetwork

In [18]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
rbf_nn = RBFNeuralNetwork(num_centers=10)
rbf_nn.fit(X_train, y_train)
y_pred = rbf_nn.predict(X_test)

weights: [[ 0.00908038  0.19805672 -0.03297598]
 [ 0.35677704  0.01142755 -0.01307715]
 [-0.00343488  0.15396572  0.09466194]
 [-0.02212228  0.00124351  0.22427974]
 [ 0.00586689  0.18609878  0.00155973]
 [-0.0113178   0.08279921  0.13535133]
 [ 0.36529275 -0.01375283  0.00424289]
 [ 0.00712486 -0.0256232   0.20066809]
 [-0.01254971  0.10396961  0.13146658]
 [ 0.05520496  0.15908812 -0.03001783]]


In [20]:
accuracy = np.mean(y_pred == y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

classes = np.unique(y_test)
for c in classes:
    classes_acc = np.mean(y_pred[y_test == c] == y_test[y_test == c])
    print(f"Accuracy class {c}: {classes_acc*100:.2f}%")

Test Accuracy: 96.67%
Accuracy class 0: 100.00%
Accuracy class 1: 100.00%
Accuracy class 2: 90.91%
